In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import keplergl
#from keplergl import KeplerGL
import matplotlib as plt
import plotly.express as px
import h3

In [2]:
intersections_geo = pd.read_csv('intersections_geography.csv')
intersections_speed = pd.read_csv('geotabs_intersection_data.csv')

# Intersections_Geo

In [3]:
intersections_geo.head()

,AsOfDate,IntersectionId,ISO_A3,ISO_3166_2,Country,State,StateId,County,CountyId,City,CityId,StreetNames,EntryCardinals,ExitCardinals,EntryStreets,ExitStreets,Latitude,Longitude
0,2021-03-01,6616316147102890000,CAN,CA-ON,Canada,Ontario,35,Toronto,3520005,NaN,NaN,"[Renforth Drive,Rathburn Road]","[E,N,S,W]","[E,N,S,W]","{\r\n ""EntryStreets"": [{\r\n ""Cardinal"": ""...","{\r\n ""ExitStreets"": [{\r\n ""Cardinal"": ""E...",43.651432,-79.579494
1,2021-03-01,-8534197810312271000,CAN,CA-ON,Canada,Ontario,35,Toronto,3520005,NaN,NaN,[The East Mall],"[E,N,S]","[N,S,W]","{\r\n ""EntryStreets"": [{\r\n ""Cardinal"": ""...","{\r\n ""ExitStreets"": [{\r\n ""Cardinal"": ""W...",43.643129,-79.558390
2,2021-03-01,1191814755289880000,CAN,CA-ON,Canada,Ontario,35,Toronto,3520005,NaN,NaN,"[Burnhamthorpe Road,The East Mall]","[E,N,S,W]","[E,N,S,W]","{\r\n ""EntryStreets"": [{\r\n ""Cardinal"": ""...","{\r\n ""ExitStreets"": [{\r\n ""Cardinal"": ""E...",43.646285,-79.560003
3,2021-03-01,-2457939225605610000,CAN,CA-ON,Canada,Ontario,35,Toronto,3520005,NaN,NaN,"[Kipling Avenue,Torlake Crescent]","[E,N,S]","[N,S,W]","{\r\n ""EntryStreets"": [{\r\n ""Cardinal"": ""...","{\r\n ""ExitStreets"": [{\r\n ""Cardinal"": ""N...",43.611565,-79.522578
4,2021-03-01,-2093932053484100000,CAN,CA-ON,Canada,Ontario,35,Toronto,3520005,NaN,NaN,"[The Queensway,Barclay Avenue]","[E,N,W]","[E,S,W]","{\r\n ""EntryStreets"": [{\r\n ""Cardinal"": ""...","{\r\n ""ExitStreets"": [{\r\n ""Cardinal"": ""S...",43.623163,-79.516799


In [4]:
#converting column headers to lowercase
intersections_geo.columns = [s.strip().lower() for s in intersections_geo.columns]

In [5]:
#dropping unneccesary columns
intersections_geo = intersections_geo.drop(columns=['asofdate','iso_a3','iso_3166_2','country','state','stateid','county','countyid','city','cityid','entrycardinals','exitcardinals','entrystreets','exitstreets'])
intersections_geo.head()

,intersectionid,streetnames,latitude,longitude
0,6616316147102890000,"[Renforth Drive,Rathburn Road]",43.651432,-79.579494
1,-8534197810312271000,[The East Mall],43.643129,-79.558390
2,1191814755289880000,"[Burnhamthorpe Road,The East Mall]",43.646285,-79.560003
3,-2457939225605610000,"[Kipling Avenue,Torlake Crescent]",43.611565,-79.522578
4,-2093932053484100000,"[The Queensway,Barclay Avenue]",43.623163,-79.516799


In [6]:
#we want to convert the latitude and longitude values to h3 hex cells

#first, we need to combine the latitude and longitude values into a coordinates column
intersections_geo['coordinates'] = intersections_geo['latitude'].astype('str') + ',' + intersections_geo['longitude'].astype('str')

#then we define the function to get h3 hex cells from coordinates data
def get_hex_cell(coordinates):
    a = float(coordinates.split(',')[0])
    b = float(coordinates.split(',')[1])
    return h3.latlng_to_cell(a,b,res=10)

#finally, create a new column for hex cells by applying the function on coordinates column
intersections_geo['hex_cell'] = intersections_geo['coordinates'].apply(get_hex_cell)

In [7]:
intersections_geo.head()

,intersectionid,streetnames,latitude,longitude,coordinates,hex_cell
0,6616316147102890000,"[Renforth Drive,Rathburn Road]",43.651432,-79.579494,"43.6514325,-79.579494",8a2b9bcf1b07fff
1,-8534197810312271000,[The East Mall],43.643129,-79.558390,"43.6431292,-79.5583904",8a2b9bce220ffff
2,1191814755289880000,"[Burnhamthorpe Road,The East Mall]",43.646285,-79.560003,"43.6462847,-79.5600029",8a2b9bce238ffff
3,-2457939225605610000,"[Kipling Avenue,Torlake Crescent]",43.611565,-79.522578,"43.6115654,-79.5225782",8a2b9bcee387fff
4,-2093932053484100000,"[The Queensway,Barclay Avenue]",43.623163,-79.516799,"43.6231631,-79.5167988",8a2b9bcee817fff


# Intersections_Speed

In [8]:
intersections_speed.head()

,DateFrom,DateTo,AggregationPeriod,AggregationPeriodUnit,AggregationPeriodFrom,IntersectionId,PercentageStopping,AvgTravelSpeed,AvgRunningSpeed,AvgTotalTimeStopped
0,2021-02-28,2021-03-13,1,hour,00:00:00.000,-9218447023040957000,0.00,41.61,41.61,0.00
1,2021-02-28,2021-03-13,1,hour,01:00:00.000,-9218447023040957000,0.26,26.24,28.73,12.63
2,2021-02-28,2021-03-13,1,hour,02:00:00.000,-9218447023040957000,0.05,33.40,33.91,3.00
3,2021-02-28,2021-03-13,1,hour,03:00:00.000,-9218447023040957000,0.00,40.79,40.79,0.00
4,2021-02-28,2021-03-13,1,hour,04:00:00.000,-9218447023040957000,0.00,47.42,47.42,0.00


In [9]:
#convert column headers to lowercase
intersections_speed.columns = [s.strip().lower() for s in intersections_speed.columns]

In [17]:
#summary statistics for dataframe
intersections_speed.describe()

,aggregationperiod,intersectionid,percentagestopping,avgtravelspeed,avgrunningspeed,avgtotaltimestopped
count,223728.0,2.237280e+05,213656.000000,213656.000000,213656.000000,213656.000000
mean,1.0,-1.050393e+17,0.287942,35.649439,38.870269,9.547358
std,0.0,5.305332e+18,0.201158,10.990329,9.574697,9.941561
min,1.0,-9.218447e+18,0.000000,4.530000,9.500000,0.000000
25%,1.0,-4.663487e+18,0.120000,27.310000,31.600000,2.770000
50%,1.0,-1.108077e+17,0.270000,35.470000,38.980000,7.130000
75%,1.0,4.629088e+18,0.430000,43.760000,45.800000,13.060000
max,1.0,9.198831e+18,1.000000,95.560000,95.910000,138.750000


In [10]:
#creaeting a pivot table of median travel speed by intersection id
intersections_speed_pivot = pd.pivot_table(intersections_speed,values='avgtravelspeed',index='intersectionid',aggfunc='median')

In [11]:
intersections_speed_pivot

,avgtravelspeed
intersectionid,
-9218447023040957000,34.880
-9215297562714759000,21.840
-9214641861738066000,51.655
-9214351989209907000,48.870
-9213702318109059000,21.935
...,...
9186244371263032000,24.325
9190058573465266000,24.700
9194870885070866000,28.025


# Merge dataframes

In [12]:
#merging the dataframes on common intersection id column
intersections_speed_processed = intersections_geo.merge(right=intersections_speed_pivot,how='inner',on='intersectionid')

In [13]:
intersections_speed_processed

,intersectionid,streetnames,latitude,longitude,coordinates,hex_cell,avgtravelspeed
0,3389382494295200000,"[Old Weston Road,Davenport Road]",43.671047,-79.462342,"43.6710467,-79.4623417",8a2b9bc096effff,22.860
1,3389382494295200000,"[Old Weston Road,Davenport Road]",43.671047,-79.462342,"43.6710467,-79.4623417",8a2b9bc096effff,22.860
2,-6671466519926600000,"[Culford Road,Lawrence Avenue West]",43.706381,-79.487640,"43.706381,-79.4876405",8a2b9bc0e20ffff,36.190
3,-6671466519926600000,"[Culford Road,Lawrence Avenue West]",43.706381,-79.487640,"43.706381,-79.4876405",8a2b9bc0e20ffff,36.190
4,9140491979021230000,"[Wynford Drive,Wynford Heights Crescent,Concor...",43.727374,-79.327757,"43.727374,-79.327757",8a2b9bd58147fff,21.105
...,...,...,...,...,...,...,...
251,-988121689216688000,"[Steeles Avenue West,Futurity Gate,Gerry Fitzg...",43.786170,-79.474730,"43.7861705,-79.47473",8a2b9bc22297fff,40.840
252,-7838963659130630000,[Birchmount Road],43.786601,-79.300638,"43.786601,-79.3006376",8a2b9bd4609ffff,44.800
253,-7838963659130630000,[Birchmount Road],43.786601,-79.300638,"43.786601,-79.3006376",8a2b9bd4609ffff,44.800
254,-8143630142906360000,"[Finch Avenue West,Milady Road]",43.747281,-79.564403,"43.7472805,-79.5644026",8a2b9bc110f7fff,25.890


In [16]:
#summary statistics for avg travel speed
intersections_speed_processed.avgtravelspeed.describe()

count    256.000000
mean      34.937246
std        9.507832
min       17.685000
25%       25.786250
50%       34.985000
75%       42.460000
max       54.720000
Name: avgtravelspeed, dtype: float64

In [93]:
intersections_speed_processed.to_csv('intersections_speed_processed.csv')